<a href="https://colab.research.google.com/github/mdeevan/LightweightFineTuning/blob/main/LightweightFineTuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

* PEFT technique: <br>
**LoRA** (Low Rank Adaptation). It decomposes a large matrix into small matrices, reducing number of parameters. It requires less memory and speeds up fine-tuning.
<br>https://huggingface.co/docs/peft/developer_guides/lora
<br>

* Model: <br>
**distilbert/distilbert-base-uncased** : It's smaller and faster then BERT and used BERT base model as a teacher.
<br>https://huggingface.co/distilbert/distilbert-base-uncased
<br>

* Evaluation approach: <br>
**seqeval** framework for sequence labeling evaluation. It evaluates the precision, recall and f1 score.
<br>https://huggingface.co/spaces/evaluate-metric/seqeval
<br>

* Fine-tuning dataset: <br>
**financial_phrasebank** based on the financial news, a multi-class-classification with three sentiments (positive, negative and neutral)
<br>https://huggingface.co/datasets/financial_phrasebank


## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

In [ ]:
!pip install accelerate


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 5.2 MB/s eta 0:00:00


In [ ]:
!kill -9 -1

In [1]:
!pip install transformers --upgrade
!pip install evaluate seqeval
!pip install peft

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 5.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.3 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=c3e090dde7aa92ad443694331ff4aea51aa4ac09d05b237c13d91cf722837608
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.9/190.9 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 11.1 MB/s eta 0:00:00


#### IMPORTS

In [2]:
from transformers import (AutoModelForSequenceClassification,
                          AutoTokenizer, DataCollatorWithPadding,
                          TrainingArguments, Trainer)
from datasets     import load_dataset

import torch
import evaluate


#### Define Variables & Load dataset

In [3]:
checkpoint = "distilbert/distilbert-base-uncased"
data_file = "financial_phrasebank"
data_file_subset = "sentences_66agree"

In [4]:
raw_dataset = load_dataset(path=data_file,
                           name=data_file_subset,
                           split="train").train_test_split(test_size=0.2,
                                                           shuffle=True,
                                                           seed=42)

raw_train = raw_dataset.pop('train')
raw_train_valid = raw_train.train_test_split(test_size=.1, shuffle=True, seed=42)
raw_dataset['train'] = raw_train_valid.pop('train')
raw_dataset['eval'] = raw_train_valid.pop('test')
raw_dataset


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/4217 [00:00<?, ? examples/s]

DatasetDict({
    test: Dataset({
        features: ['sentence', 'label'],
        num_rows: 844
    })
    train: Dataset({
        features: ['sentence', 'label'],
        num_rows: 3035
    })
    eval: Dataset({
        features: ['sentence', 'label'],
        num_rows: 338
    })
})

In [5]:
# raw_dataset

In [6]:
labels = raw_dataset["train"].features['label'].names
labels

['negative', 'neutral', 'positive']

In [7]:
label2id = {l:i for i, l in enumerate(labels)}
id2label = {i:l for i, l in enumerate(labels)}

In [8]:
input_max_length = max([len(s) for s in raw_dataset['train']['sentence']])
input_max_length

315

In [9]:
print(label2id)
print(id2label)
print(len(label2id))

{'negative': 0, 'neutral': 1, 'positive': 2}
{0: 'negative', 1: 'neutral', 2: 'positive'}
3


In [18]:
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint,
                                                           num_labels = len(label2id),
                                                           id2label=id2label,
                                                           label2id=label2id)

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

print(device)

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


cuda


In [19]:
print("model     = ", model)
print("tokenizer = ", tokenizer)

model     =  DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inp

In [20]:
# data_collator

In [21]:
def tokenize_function(data):
    return tokenizer(data['sentence'],
#                      max_length=input_max_length,
                     truncation=True,
#                      padding='max_length'
                    )


tokenized_dataset = raw_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/844 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/3035 [00:00<?, ? examples/s]

Map:   0%|          | 0/338 [00:00<?, ? examples/s]

In [22]:
tokenized_dataset

DatasetDict({
    test: Dataset({
        features: ['sentence', 'label', 'input_ids', 'attention_mask'],
        num_rows: 844
    })
    train: Dataset({
        features: ['sentence', 'label', 'input_ids', 'attention_mask'],
        num_rows: 3035
    })
    eval: Dataset({
        features: ['sentence', 'label', 'input_ids', 'attention_mask'],
        num_rows: 338
    })
})

In [23]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer,
                                        padding=True,
#                                         padding='max_length',
#                                         max_length=input_max_length)
                                       )

In [24]:
data_collator

DataCollatorWithPadding(tokenizer=DistilBertTokenizerFast(name_or_path='distilbert/distilbert-base-uncased', vocab_size=30522, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}, padding=True, max_length=None, pad_to_mul

In [25]:
[len(x) for x in tokenized_dataset['train'][:10]['sentence']]

[120, 75, 152, 46, 152, 90, 50, 167, 160, 49]

In [26]:
[len(x) for x in tokenized_dataset['train'][:10]['input_ids']]

[22, 19, 38, 12, 35, 20, 14, 57, 42, 16]

In [27]:
sample_start = 20
sample_count = 10

samples = tokenized_dataset['train'][sample_start : sample_start+sample_count]

samples = {k: v for k, v in samples.items() if k not in ['sentence', 'label']}
print([len(x) for x in samples['input_ids']])



[26, 26, 33, 28, 25, 29, 10, 31, 41, 17]


In [28]:
# data_collator([samples[i] for i in range(2)])

In [29]:
samples.keys()

dict_keys(['input_ids', 'attention_mask'])

In [30]:
batch = data_collator(samples ).to(device)
# batch
# {k: v.shape for k, v in batch.items()}


In [31]:
{k: v.shape for k, v in batch.items()}

{'input_ids': torch.Size([10, 41]), 'attention_mask': torch.Size([10, 41])}

In [32]:
output = model(**batch).logits
# output

In [33]:
  predictions=torch.argmax(output, dim=1).cpu().numpy()
  predictions

array([1, 1, 1, 1, 1, 1, 1, 2, 1, 1])

In [34]:
references=tokenized_dataset['train']['label'][sample_start:sample_start+sample_count]
references

[1, 2, 2, 1, 2, 0, 1, 1, 1, 2]

In [35]:
batch.keys()

dict_keys(['input_ids', 'attention_mask'])

In [37]:
# https://www.evidentlyai.com/classification-metrics/multi-class-metrics

accuracy  = evaluate.load('accuracy')
f1        = evaluate.load('f1')
precision = evaluate.load('precision')
recall    = evaluate.load('recall')

In [38]:
import numpy as np
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    return accuracy.compute(predictions=predictions, references=labels)

In [39]:


clf =  evaluate.combine(["accuracy",'f1','precision','recall'])
accuracy_metric = accuracy.compute (predictions = predictions, references  = references )
f1_metric       = f1.compute       (predictions = predictions, references  = references,  average = "macro")
precision_metric= precision.compute(predictions = predictions, references  = references,   average = "macro", zero_division=0)
recall_metric   = recall.compute   (predictions = predictions, references  = references,  average = "macro")


print(accuracy_metric)
print(f1_metric)
print(precision_metric)
print(recall_metric)

{'accuracy': 0.4}
{'f1': 0.19047619047619047}
{'precision': 0.14814814814814814}
{'recall': 0.26666666666666666}


## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

In [ ]:
# !pip install peft

In [ ]:
from peft import LoraConfig, get_peft_model, TaskType


In [ ]:
config = LoraConfig()
peft_config = LoraConfig(task_type=TaskType.SEQ_CLS ,
                         inference_mode=False,
                         r=64,
                         lora_alpha=32,
                         lora_dropout=0.1,
                         target_modules=['q_lin','v_lin', 'k_lin'],
                         )


In [ ]:
peft_model = get_peft_model(model, peft_config)

In [ ]:

peft_model.print_trainable_parameters()

trainable params: 2,362,371 || all params: 69,318,150 || trainable%: 3.4080121872842826


In [ ]:
#

In [ ]:
Training_Arguments = TrainingArguments(
    per_device_train_batch_size = 32,
    per_device_eval_batch_size  = 32,
    output_dir                  = "my_Multi_label_Classifier2",
    learning_rate               = 2e-5,
    num_train_epochs            = 20,
    weight_decay                = 0.01,
    save_strategy               = 'epoch',
    evaluation_strategy         = 'epoch',
    deepspeed                   = False,
    load_best_model_at_end      = True)

In [ ]:
tokenized_dataset['train'].features

{'sentence': Value(dtype='string', id=None),
 'label': ClassLabel(names=['negative', 'neutral', 'positive'], id=None),
 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None),
 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None)}

In [ ]:
tokenized_dataset['train'].rename_column('label','labels')

Dataset({
    features: ['sentence', 'labels', 'input_ids', 'attention_mask'],
    num_rows: 3035
})

In [ ]:
trainer = Trainer(
                  model=model,
                  args=Training_Arguments,
                  train_dataset=tokenized_dataset['train'],
                  eval_dataset =tokenized_dataset['eval'],
                  compute_metrics=compute_metrics,
                  tokenizer=tokenizer,
                  data_collator=data_collator
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.337737,0.860947
2,No log,0.324453,0.857988
3,No log,0.313892,0.849112
4,No log,0.303739,0.869822
5,No log,0.291806,0.863905
6,0.313000,0.284938,0.857988
7,0.313000,0.278724,0.875740
8,0.313000,0.269923,0.878698
9,0.313000,0.267660,0.887574
10,0.313000,0.263403,0.887574


TrainOutput(global_step=1900, training_loss=0.25973279049521997, metrics={'train_runtime': 309.7937, 'train_samples_per_second': 195.937, 'train_steps_per_second': 6.133, 'total_flos': 1083856880276820.0, 'train_loss': 0.25973279049521997, 'epoch': 20.0})

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# model.save_pretrained("fineTunedMultiLabelClassifier")

In [ ]:
trainer.save_model("/content/drive/MyDrive/ftMLC")

In [ ]:
tokenized_dataset['train'].features

{'sentence': Value(dtype='string', id=None),
 'label': ClassLabel(names=['negative', 'neutral', 'positive'], id=None),
 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None),
 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None)}

## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.

In [11]:
text = "This was a masterpiece. Not completely faithful to the books, but enthralling from beginning to end. Might be my favorite of the three."

In [14]:
saved_checkpoint = '/content/drive/MyDrive/ftMLC'
tokenizer2 = AutoTokenizer.from_pretrained(saved_checkpoint)
inputs = tokenizer2(text, return_tensors='pt')

In [15]:
from transformers import AutoModelForSequenceClassification

model2 = AutoModelForSequenceClassification.from_pretrained(saved_checkpoint)

Some weights of the model checkpoint at /content/drive/MyDrive/ftMLC were not used when initializing DistilBertForSequenceClassification: ['classifier.modules_to_save.default.bias', 'classifier.modules_to_save.default.weight', 'classifier.original_module.bias', 'classifier.original_module.weight', 'distilbert.transformer.layer.0.attention.k_lin.base_layer.bias', 'distilbert.transformer.layer.0.attention.k_lin.base_layer.weight', 'distilbert.transformer.layer.0.attention.k_lin.lora_A.default.weight', 'distilbert.transformer.layer.0.attention.k_lin.lora_B.default.weight', 'distilbert.transformer.layer.0.attention.q_lin.base_layer.bias', 'distilbert.transformer.layer.0.attention.q_lin.base_layer.weight', 'distilbert.transformer.layer.0.attention.q_lin.lora_A.default.weight', 'distilbert.transformer.layer.0.attention.q_lin.lora_B.default.weight', 'distilbert.transformer.layer.0.attention.v_lin.base_layer.bias', 'distilbert.transformer.layer.0.attention.v_lin.base_layer.weight', 'distilbert

In [16]:
with torch.no_grad():
  logits = model2(**inputs).logits

In [40]:
prediction = np.argmax(logits, axis=-1).numpy()
print(model2.config.id2label[prediction[0]])

neutral


In [41]:
print(prediction[0])

1


In [48]:
tokenized_dataset['test'].features

{'sentence': Value(dtype='string', id=None),
 'label': ClassLabel(names=['negative', 'neutral', 'positive'], id=None),
 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None),
 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None)}

In [49]:

test_set = tokenized_dataset['test']
# test_set = {k: v for k, v in tokenized_dataset['test'].items() if k not in ['sentence', 'label']}
# print([len(x) for x in samples['input_ids']])



In [ ]:
test_set[:]


In [56]:
Training_Arguments2 = TrainingArguments(
    per_device_train_batch_size = 32,
    per_device_eval_batch_size  = 32,
    output_dir                  = "evaluate",
    learning_rate               = 2e-5,
    num_train_epochs            = 20,
    weight_decay                = 0.01,
    save_strategy               = 'epoch',
    evaluation_strategy         = 'epoch',
    deepspeed                   = False,
    load_best_model_at_end      = True)

In [57]:
trainer2 = Trainer(
                  model=model2,
                  args=Training_Arguments2,
                  train_dataset=tokenized_dataset['train'],
                  eval_dataset =tokenized_dataset['test'],
                  compute_metrics=compute_metrics,
                  tokenizer=tokenizer2,
                  data_collator=data_collator
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [58]:
print(trainer2.evaluate())

{'eval_loss': 1.0536293983459473, 'eval_accuracy': 0.5995260663507109, 'eval_runtime': 1.8613, 'eval_samples_per_second': 453.435, 'eval_steps_per_second': 14.506}


In [60]:

def evaluate_samples(model=model, ds=tokenized_dataset['train'], sample_start=0, sample_count=10):
  samples = ds[sample_start : sample_start+sample_count]

  samples = {k: v for k, v in samples.items() if k not in ['sentence', 'label']}

  batch = data_collator(samples ).to(device)

  output = model(**batch).logits

  predictions=torch.argmax(output, dim=1).cpu().numpy()

  return predictions


In [84]:
print(evaluate_samples(model2, tokenized_dataset['test'], 744, 100))

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]


In [70]:
tokenized_dataset['test'] #.features

Dataset({
    features: ['sentence', 'label', 'input_ids', 'attention_mask'],
    num_rows: 844
})

In [86]:
print(evaluate_samples(model, tokenized_dataset['train'], 744, 100))

[2 1 1 1 1 1 2 1 1 1 1 1 1 2 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 2 1 2 1 1 1 1 1
 1 1 1 1 2 1 2 1 1 1 2 1 1 1 1 1 1 2 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 2 1 1 1 1 2 1 1 1 2 1 1 1 1 1 1 2 1 1]


In [94]:
import pandas as pd

In [118]:
df = pd.DataFrame()
df['label'] = tokenized_dataset['eval'][:]['label']
df['val'] = 1

df.groupby(['label']).sum(['val'])

,val
label,
0,39
1,200
2,99


In [114]:
df.groupby(['label']).sum(['val'])

,val
label,
0,107
1,506
2,231
